In [1]:
import re
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.types import NVARCHAR, Float, DECIMAL

raw_data = pd.read_csv("raw_data.csv", low_memory=False)

# AE_bankruptcy_list 生成

In [2]:
AE_data = pd.read_csv("AE_badcompany_pred.csv")
AE_data["time_q"] = AE_data["time_q"].apply(lambda x: x.replace("Q4","/12").replace("Q3","/9").replace("Q2","/6").replace("Q1","/3"))
AE_data = AE_data[AE_data.time_q.isin(["2021/3","2020/12","2020/9","2020/6","2020/3","2019/12","2019/9","2019/6","2019/3"])]
AE_data['datetime'] = pd.to_datetime(AE_data['time_q'], format="%Y/%m")
AE_data.sort_values(by=['time_q']).drop_duplicates(subset=['company'], keep='last',inplace=True)

In [3]:
all_data_list = raw_data.drop_duplicates(subset=['company'], keep='last')
AE_data=AE_data.merge(all_data_list[["company","abbr","TSE_industry_new"]],how="left",on="company")
AE_data = AE_data[AE_data["company"].str.match("^(\d{4})$")==True]
AE_data["TSE_industry_new"] = AE_data["TSE_industry_new"].apply(lambda x: re.sub("[0-9a-zA-Z]","",x))
AE_data=AE_data[['company','abbr','TSE_industry_new']]

In [4]:
# sql 大小寫不分 => 欄位名稱全部轉小寫
# AE_data.columns = [x.lower() for x in AE_data.columns]
dtypedict = dict()
for col, dtype in AE_data.dtypes.iteritems():
    dtypedict[col] = DECIMAL(20,10) if str(dtype)=="float64" or str(dtype)=="int64" else NVARCHAR(length=50)

engine = create_engine("mysql://{0}:{1}@{2}:{3}/{4}?charset=utf8".format("root", "0936615983", "127.0.0.1", "3306", "finance"))
con = engine.connect()
AE_data.to_sql(name='ae_bankruptcy_list', con=con, if_exists='replace', index=False, dtype=dtypedict)

In [10]:
AE_data['company'].tolist()

['1104',
 '1201',
 '1201',
 '1210',
 '1225',
 '1236',
 '1258',
 '1316',
 '1326',
 '1341',
 '1416',
 '1417',
 '1440',
 '1446',
 '1452',
 '1452',
 '1471',
 '1507',
 '1529',
 '1560',
 '1592',
 '1734',
 '1735',
 '1781',
 '1810',
 '2012',
 '2013',
 '2014',
 '2020',
 '2025',
 '2032',
 '2059',
 '2066',
 '2325',
 '2327',
 '2332',
 '2349',
 '2358',
 '2368',
 '2390',
 '2404',
 '2414',
 '2419',
 '2433',
 '2434',
 '2434',
 '2439',
 '2456',
 '2461',
 '2478',
 '2480',
 '2484',
 '2493',
 '2496',
 '2535',
 '2542',
 '2546',
 '2605',
 '2607',
 '2607',
 '2608',
 '2613',
 '2745',
 '2754',
 '2756',
 '2905',
 '2906',
 '2926',
 '3010',
 '3019',
 '3029',
 '3046',
 '3049',
 '3058',
 '3105',
 '3105',
 '3114',
 '3122',
 '3131',
 '3163',
 '3217',
 '3227',
 '3230',
 '3260',
 '3266',
 '3297',
 '3303',
 '3312',
 '3362',
 '3390',
 '3511',
 '3518',
 '3521',
 '3523',
 '3530',
 '3591',
 '3592',
 '3632',
 '3685',
 '3705',
 '3712',
 '4105',
 '4116',
 '4139',
 '4147',
 '4150',
 '4160',
 '4168',
 '4173',
 '4188',
 '4190',
 

# stock_list table生成

In [5]:
# raw_data = pd.read_csv("raw_data.csv")
# 找出公司股票
raw_data.company = raw_data.company.astype("str")
company_data = raw_data[raw_data["company"].str.match("^(\d{4})$")==True]
company_data["time_q"] = company_data["time_q"].apply(lambda x: x.replace("Q4","/12").replace("Q3","/9").replace("Q2","/6").replace("Q1","/3"))
# 篩選需要欄位
company_data = company_data[company_data.time_q.isin(["2021/3","2020/12","2020/9","2020/6","2020/3","2019/12","2019/9","2019/6","2019/3"])]
rf_std_features = ["Retained_earnings_assets","negative_two_years","earning_before_tax_margin","Income_Before_Tax_and_Interests_assets","Operating_margin","Debt_Equity_Ratio","Net_Asset_Value_per_Share_F","ration_of_interest_expense","l4","i69","net_income_change_rate","l3","l17","Interest_Coverage_Ratio","current_liab_current_assets","debt_ratio"]
other_features = ["company","time_q","abbr","TSE_industry_new"]
all_features = rf_std_features + other_features
company_data = company_data[all_features]

<ipython-input-5-227f05c27b83>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  company_data["time_q"] = company_data["time_q"].apply(lambda x: x.replace("Q4","/12").replace("Q3","/9").replace("Q2","/6").replace("Q1","/3"))


In [6]:
company_data["TSE_industry_new"].fillna("未分類",inplace=True)
company_data["TSE_industry_new"] = company_data["TSE_industry_new"].apply(lambda x: re.sub("[0-9a-zA-Z]","",x))
unique_company_data = company_data[["TSE_industry_new", "company", "abbr"]].drop_duplicates()
unique_company_data = unique_company_data[~unique_company_data['company'].isin(AE_data['company'].tolist())]

In [7]:
dtypedict = dict()
for col, dtype in unique_company_data.dtypes.iteritems():
    dtypedict[col] = DECIMAL(20,10) if str(dtype)=="float64" or str(dtype)=="int64" else NVARCHAR(length=50)

In [8]:
engine = create_engine("mysql://{0}:{1}@{2}:{3}/{4}?charset=utf8".format("root", "0936615983", "127.0.0.1", "3306", "finance"))
con = engine.connect()
unique_company_data.to_sql(name='stock_list', con=con, if_exists='replace', index=False, dtype=dtypedict)

# two_year table生成

In [ ]:
raw_data = pd.read_csv("raw_data.csv")
# 找出公司股票
raw_data.company = raw_data.company.astype("str")
company_data = raw_data[raw_data["company"].str.match("^(\d{4})$")==True]
company_data["time_q"] = company_data["time_q"].apply(lambda x: x.replace("Q4","/12").replace("Q3","/9").replace("Q2","/6").replace("Q1","/3"))
# 篩選需要欄位
rf_std_features = ["Retained_earnings_assets","negative_two_years","earning_before_tax_margin","Income_Before_Tax_and_Interests_assets","Operating_margin","Debt_Equity_Ratio","Net_Asset_Value_per_Share_F","ration_of_interest_expense","l4","i69","net_income_change_rate","l3","l17","Interest_Coverage_Ratio","current_liab_current_assets","debt_ratio"]
other_features = ["company","time_q","abbr","TSE_industry_new"]
company_data['datetime'] = pd.to_datetime(company_data['time_q'], format="%Y/%m")

all_features = rf_std_features + other_features
company_data = company_data[all_features]

In [ ]:
company_data["TSE_industry_new"].fillna("未分類",inplace=True)
company_data["TSE_industry_new"] = company_data["TSE_industry_new"].apply(lambda x: re.sub("[0-9a-zA-Z]","",x))

In [ ]:
#篩選年分
company_data = company_data[company_data.time_q.isin(["2021/3","2020/12","2020/9","2020/6","2020/3","2019/12","2019/9","2019/6","2019/3"])]

In [ ]:
# fill missing value with zero
company_data.fillna(0,inplace=True)

In [ ]:
dtypedict = dict()
for col, dtype in company_data.dtypes.iteritems():
    dtypedict[col] = DECIMAL(20,10) if str(dtype)=="float64" or str(dtype)=="int64" else NVARCHAR(length=50)

In [ ]:
engine = create_engine("mysql://{0}:{1}@{2}:{3}/{4}?charset=utf8".format("root", "0936615983", "127.0.0.1", "3306", "finance"))
con = engine.connect()
company_data.to_sql(name='two_year', con=con, if_exists='replace', index=False, dtype=dtypedict)